# Práctica: Análisis comparativo de tecnologías usando RAG

Objetivo de la práctica

Cargar varios archivos de documentos de diferentes fuentes (por ejemplo, artículos científicos, noticias, informes técnicos) y generar embeddings para cada uno.

Implementar un retriever que busque documentos relevantes según consultas comparativas o de análisis.

Implementar un generator que combine los documentos recuperados y genere un resumen que permita comparar o evaluar la información.

Probar el sistema con preguntas de análisis crítico y comparativo.

1. Preparación de datos

- La carpeta Data contiene archivos .csv o .txt de diferentes fuentes sobre energía renovable (solar, eólica, almacenamiento).

- Generar embeddings para cada documento usando un modelo de embeddings (por ejemplo, text-embedding-3-small).

In [ ]:
# Paso 1: Instalar e importar librerías
# Ejecutar esta celda primero para instalar las dependencias necesarias
# !pip install sentence-transformers scikit-learn -q

import os
import glob
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

print("📦 Importando librerías...")

# Cargar modelo de embeddings LOCAL (100% GRATIS, sin API key necesaria)
print("\n📥 Cargando modelo de embeddings local (Sentence Transformers)...")
embedding_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print("✅ Modelo de embeddings cargado")

print("\n✅ Inicialización completada")

⚠️  No se encontró OPENAI_API_KEY en las variables de entorno
✓ Librerías importadas correctamente
✓ Cliente OpenAI configurado
✓ Librerías importadas correctamente
✓ Cliente OpenAI configurado


In [ ]:
# Paso 2: Cargar documentos desde la carpeta Data
data_folder = "Data"
documents = []

print("📂 Cargando documentos desde carpeta Data...")
txt_files = glob.glob(os.path.join(data_folder, "ejer4_*.txt"))

for file_path in txt_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        source_name = os.path.basename(file_path).replace('.txt', '').replace('ejer4_', '')
        documents.append({
            'source': source_name,
            'content': content,
            'file_path': file_path
        })

print(f"\n✅ Se cargaron {len(documents)} documentos:")
for doc in documents:
    print(f"   • {doc['source']}: {len(doc['content'])} caracteres")

In [ ]:
# Paso 3: Generar embeddings para cada documento
def generate_embedding(text):
    """Genera embedding usando Sentence Transformers (modelo local)"""
    return embedding_model.encode(text, convert_to_numpy=True)

print("🔄 Generando embeddings (esto puede tardar unos segundos)...")
for i, doc in enumerate(documents, 1):
    doc['embedding'] = generate_embedding(doc['content'])
    print(f"   ✓ [{i}/{len(documents)}] {doc['source']}")

print(f"\n✅ Embeddings generados exitosamente")
print(f"📊 Dimensión de cada embedding: {len(documents[0]['embedding'])} dimensiones")

2. Implementación del Retriever

El retriever debe permitir:

- Buscar documentos relevantes por tema.

- Buscar documentos relevantes por comparación, es decir, identificar información sobre dos tecnologías o conceptos diferentes.

- Utilizar búsqueda semántica con embeddings para medir similitud.

In [ ]:
# Implementación del Retriever
class SemanticRetriever:
    """
    Retriever que busca documentos relevantes usando similitud semántica
    """
    def __init__(self, documents):
        self.documents = documents
        self.embeddings = np.array([doc['embedding'] for doc in documents])
    
    def retrieve(self, query, top_k=3):
        """
        Recupera los top_k documentos más relevantes para una consulta
        """
        # Generar embedding de la consulta
        query_embedding = generate_embedding(query).reshape(1, -1)
        
        # Calcular similitud coseno
        similarities = cosine_similarity(query_embedding, self.embeddings)[0]
        
        # Obtener índices de los más similares
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        
        # Retornar resultados
        results = []
        for idx in top_indices:
            results.append({
                'document': self.documents[idx],
                'similarity_score': similarities[idx]
            })
        return results
    
    def retrieve_for_comparison(self, query, technologies, top_k=2):
        """
        Recupera documentos específicos para comparación de tecnologías
        """
        results = {}
        
        for tech in technologies:
            # Crear consulta específica para cada tecnología
            tech_query = f"{query} {tech}"
            tech_embedding = generate_embedding(tech_query).reshape(1, -1)
            
            # Calcular similitud
            similarities = cosine_similarity(tech_embedding, self.embeddings)[0]
            top_indices = np.argsort(similarities)[-top_k:][::-1]
            
            results[tech] = []
            for idx in top_indices:
                results[tech].append({
                    'document': self.documents[idx],
                    'similarity_score': similarities[idx]
                })
        
        return results

# Crear instancia del retriever
retriever = SemanticRetriever(documents)
print("✅ Retriever creado e inicializado correctamente")

In [ ]:
# Prueba del retriever con una consulta simple
test_query = "energía solar y paneles fotovoltaicos"
print(f"🔍 Probando retriever con: '{test_query}'")
print("="*80)

results = retriever.retrieve(test_query, top_k=3)

for i, result in enumerate(results, 1):
    print(f"\n{i}. 📄 Fuente: {result['document']['source']}")
    print(f"   📊 Similitud: {result['similarity_score']:.4f}")
    print(f"   📝 Preview: {result['document']['content'][:120]}...")

print("\n" + "="*80)
print("✅ Retriever funcionando correctamente")

3. Implementación del Generator

El generator debe:

   - Tomar los documentos recuperados y resumir los puntos clave.

   - Comparar tecnologías, métodos o resultados si la consulta lo solicita.

   - Generar un texto coherente y estructurado, por ejemplo:

       Introducción breve.

       Comparación de características, ventajas y desventajas.

       Conclusión resumida.

In [ ]:
# Implementación del Generator (versión simplificada sin LLM)
class ComparativeGenerator:
    """
    Generador que crea análisis comparativos mostrando documentos recuperados
    """
    
    def generate_summary(self, query, retrieved_docs):
        """Genera un resumen mostrando los documentos recuperados"""
        output = f"\n{'='*80}\n"
        output += f"RESUMEN SOBRE: {query}\n"
        output += f"{'='*80}\n\n"
        
        for i, doc_result in enumerate(retrieved_docs, 1):
            doc = doc_result['document']
            output += f"📄 DOCUMENTO {i}: {doc['source'].upper()}\n"
            output += f"📊 Similitud: {doc_result['similarity_score']:.4f}\n"
            output += f"{'─'*80}\n"
            output += f"{doc['content']}\n"
            output += f"{'─'*80}\n\n"
        
        return output
    
    def generate_comparison(self, query, comparison_docs):
        """Genera una comparación mostrando documentos por tecnología"""
        output = f"\n{'='*80}\n"
        output += f"COMPARACIÓN: {query}\n"
        output += f"{'='*80}\n\n"
        
        for tech, docs in comparison_docs.items():
            output += f"🔬 TECNOLOGÍA: {tech.upper()}\n"
            output += f"{'='*80}\n"
            
            for i, doc_result in enumerate(docs, 1):
                doc = doc_result['document']
                output += f"\n📄 Documento {i}: {doc['source']}\n"
                output += f"📊 Similitud: {doc_result['similarity_score']:.4f}\n"
                output += f"{'─'*80}\n"
                output += f"{doc['content']}\n"
                output += f"{'─'*80}\n"
            
            output += "\n"
        
        return output

# Crear instancia del generador
generator = ComparativeGenerator()
print("✅ Generator creado e inicializado (versión simplificada sin LLM)")

In [ ]:
# Sistema RAG completo
def rag_system(query, comparison_mode=False, technologies=None, top_k=3):
    """
    Sistema RAG completo: Retrieval + Generation (versión simplificada)
    """
    print(f"\n{'='*80}")
    print(f"📝 CONSULTA: {query}")
    print('='*80)
    
    if comparison_mode and technologies:
        print(f"🔍 Modo: COMPARACIÓN entre {' vs '.join(technologies)}")
        print("\nRecuperando documentos relevantes...")
        
        retrieved = retriever.retrieve_for_comparison(query, technologies, top_k=top_k)
        
        print("\n📄 Documentos recuperados por tecnología:")
        for tech, docs in retrieved.items():
            print(f"\n  {tech.upper()}:")
            for i, doc_result in enumerate(docs, 1):
                print(f"    {i}. {doc_result['document']['source']} (similitud: {doc_result['similarity_score']:.4f})")
        
        print("\n📝 Generando análisis comparativo...")
        response = generator.generate_comparison(query, retrieved)
        
    else:
        print(f"🔍 Modo: RESUMEN GENERAL")
        print("\nRecuperando documentos relevantes...")
        
        retrieved = retriever.retrieve(query, top_k=top_k)
        
        print("\n📄 Documentos recuperados:")
        for i, doc_result in enumerate(retrieved, 1):
            print(f"  {i}. {doc_result['document']['source']} (similitud: {doc_result['similarity_score']:.4f})")
        
        print("\n📝 Generando resumen...")
        response = generator.generate_summary(query, retrieved)
    
    print('='*80)
    return response

print("✅ Sistema RAG completo configurado y listo para usar")

4. Consultas de prueba

- “Comparar eficiencia entre células solares de perovskita y de silicio.”

- “Ventajas y desventajas del almacenamiento en baterías vs. hidrógeno.”

- “Últimas innovaciones en energía eólica frente a energía solar.”

In [ ]:
# CONSULTA 1: Comparar eficiencia entre células solares de perovskita y de silicio

query1 = "Comparar eficiencia entre células solares de perovskita y de silicio"
response1 = rag_system(
    query=query1,
    comparison_mode=True,
    technologies=["perovskita", "silicio"],
    top_k=2
)

print("\n📊 RESPUESTA GENERADA:\n")
print(response1)
print("\n" + "="*80)

In [ ]:
# CONSULTA 2: Ventajas y desventajas del almacenamiento en baterías vs. hidrógeno

query2 = "Ventajas y desventajas del almacenamiento en baterías vs. hidrógeno"
response2 = rag_system(
    query=query2,
    comparison_mode=True,
    technologies=["baterías", "hidrógeno"],
    top_k=2
)

print("\n📊 RESPUESTA GENERADA:\n")
print(response2)
print("\n" + "="*80)

In [ ]:
# CONSULTA 3: Últimas innovaciones en energía eólica frente a energía solar

query3 = "Últimas innovaciones en energía eólica frente a energía solar"
response3 = rag_system(
    query=query3,
    comparison_mode=True,
    technologies=["eólica", "solar"],
    top_k=2
)

print("\n📊 RESPUESTA GENERADA:\n")
print(response3)
print("\n" + "="*80)

## Prueba adicional: Consulta sin modo comparación

Vamos a probar también el sistema con una consulta simple (sin comparación) para ver cómo funciona el modo de resumen general.

In [ ]:
# CONSULTA ADICIONAL: Análisis general sobre almacenamiento de energía

query_extra = "¿Cuáles son los principales desafíos del almacenamiento de energía renovable?"
response_extra = rag_system(
    query=query_extra,
    comparison_mode=False,
    top_k=3
)

print("\n📊 RESPUESTA GENERADA:\n")
print(response_extra)
print("\n" + "="*80)